### Librerías

In [3]:
import numpy as np
import warnings
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, BatchNormalization, Conv2DTranspose, UpSampling2D, Flatten, Activation, Reshape
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import RMSprop, Adam, Adadelta 
from google.colab import drive
%matplotlib inline

In [4]:
warnings.filterwarnings("ignore", category=FutureWarning)

Acceder a los datos de Google Drive

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


Cargar datos y establecer (n_muestras, 224, 224, 1) como "shape".

In [6]:
images = "/content/drive/MyDrive/mamma.npy"
np_images = np.load(images, allow_pickle=True)
normalized_images = np_images/255
normalized_images = np.reshape(normalized_images,(normalized_images.shape[0],224,224,1))

#### Generador

In [7]:
def generator_architecture():
    noise_shape = (32,)
    
    model = Sequential()
    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod((224,224,1)), activation="tanh"))
    model.add(Reshape((224,224,1)))
    
    model.summary()
    
    noise = Input(shape=noise_shape)
    fake_img = model(noise)
    
    return Model(noise, fake_img)

In [8]:
generator = generator_architecture()
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               8448      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512)               0         
                                                        

### Discriminador

In [9]:
def discriminator_architecture():
    model = Sequential()
    
    model.add(Flatten(input_shape=(224,224,1)))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation="sigmoid"))
    model.summary()
    
    img = Input(shape=(224,224,1))
    validity = model(img)
    
    return Model(img, validity)

In [10]:
discriminator = discriminator_architecture()
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense_4 (Dense)             (None, 512)               25690624  
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 dropout_4 (Dropout)         (None, 256)              

In [11]:
discriminator.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.0004, decay=6e-8, clipvalue=1.0), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [12]:
noise = Input(shape=(32,))
synthetic_img = generator(noise)

In [13]:
discriminator.trainable = False

In [14]:
discriminator_pred = discriminator(synthetic_img)
adversarial_model = Model(noise, discriminator_pred)

In [15]:
adversarial_model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.0004, decay=3e-8, clipvalue=1.0), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Guardar modelo e imágenes creadas por cada "x" epochs que se establezcan.

In [16]:
GENERATOR_PATH = "/content/drive/MyDrive/generator_model_DCGAN/"
IMAGES = "/content/drive/MyDrive/images/"

### Entrenamiento de la red

In [19]:
def training_procedure(epochs=4000, batch=32):
    
    discriminator_stats, adversarial_stats = [], []
    discriminator_loss , discriminator_acc = 0, 0
    adversarial_loss, adversarial_acc = 0, 0

    for epoch in range(1, epochs+1):
        real_imgs = np.reshape(normalized_images[np.random.choice(normalized_images.shape[0],batch,replace=False)],(batch,224,224,1))
        fake_imgs = generator.predict(np.random.uniform(-1.0, 1.0, size=[batch, 32]))
    
        real_and_fake_imgs = np.concatenate((real_imgs,fake_imgs))
        
        discriminator_labels = np.ones([2*batch,1])
        discriminator_labels[batch:,:] = 0
        
        discriminator_stats.append(discriminator.train_on_batch(real_and_fake_imgs,discriminator_labels))
        discriminator_loss += discriminator_stats[-1][0]
        discriminator_acc += discriminator_stats[-1][1]
        
        noise = np.random.uniform(-1.0, 1.0, size=[batch, 32])
        y = np.ones([batch,1])

        adversarial_stats.append(adversarial_model.train_on_batch(noise,y)) 
        adversarial_loss += adversarial_stats[-1][0]
        adversarial_acc += adversarial_stats[-1][1]
             
        print("%d: [D loss: %f, acc: %f]" % (epoch, discriminator_loss/epoch, discriminator_acc/epoch))

        if (epoch+1)%100 == 0: 

          noise = np.random.uniform(-1.0, 1.0, size=[16, 32])
          gen_imgs = generator.predict(noise)
          plt.figure(figsize=(7,7))

          for k in range(gen_imgs.shape[0]):
            plt.subplot(4, 4, k+1)
            plt.imshow(gen_imgs[k, :, :, 0], cmap='gray')
            plt.axis('off')
            plt.savefig(IMAGES+'epoch%d.png' % (epoch))
 
          plt.tight_layout()
          plt.show()

    generator.save(GENERATOR_PATH+'gan_generator_%d.h5' % (epoch))
    
    return discriminator_stats

In [ ]:
discriminator_metrics, adversarial_metrics = training_procedure()